In [55]:
import sys, os, boto3
import xml.etree.ElementTree as ET

sys.path.insert(0, "../src")
import GOSTrocks.dataMisc as dMisc

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
s3_bucket = 'wbg-geography01'
s3_prefix = 'FATHOM/v2023/'
s3_out = os.path.join('s3://',s3_bucket, s3_prefix)

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(s3_bucket)

# Find all files already copied
all_folders = []
for obj in my_bucket.objects.filter(Prefix=s3_prefix):
    all_folders.append(obj.key.split("/")[-2])
all_folders
all_folders = list(set(all_folders))

['GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0',
 'GLOBAL-1ARCSEC-NW_OFFSET-1

In [8]:
# build list of rasters for generating VRT
local_path = os.path.join("v2023","GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0")
in_folder = os.path.join(template_folder, local_path)
all_tiffs = [f"{local_path}/{x}" for x in os.listdir(in_folder)]

with open(os.path.join(template_folder, "s3_tiffs.txt"), 'w') as out:
    for p in all_tiffs:
        out.write(f'{p}\n')

In [41]:
template_folder = '/home/wb411133/temp'
coastal_template = os.path.join(template_folder, "GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0.0.vrt")
other_template = os.path.join(template_folder,   "GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt")

class generate_vrt_from_template():
    def __init__(self, template_vrt, new_dataset):
        self.template_vrt = template_vrt
        self.new_dataset = new_dataset
        self.old_dataset = os.path.basename(template_vrt)[:-4]
        self.new_vrt = self.template_vrt.replace(self.old_dataset, self.new_dataset)

    def update_vrt(self):
        tree = ET.parse(self.template_vrt)
        root = tree.getroot()

        for child in root.iter("SourceFilename"):
            child.text = child.text.replace(self.old_dataset, self.new_dataset)

        tree.write(self.new_vrt, xml_declaration=False)

'''
new_ds = "GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0"
xx = generate_vrt_from_template(template_vrt, new_ds)
xx.update_vrt()
'''

'\nnew_ds = "GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0"\nxx = generate_vrt_from_template(template_vrt, new_ds)\nxx.update_vrt()\n'

In [43]:
# Create template vrts
for new_ds in all_folders:
    flood_type = new_ds.split("-")[4]
    vrt_template = other_template
    if flood_type == "COASTAL":
        vrt_template = coastal_template
    xx = generate_vrt_from_template(vrt_template, new_ds)
    xx.update_vrt()

In [ ]:
# aws s3 cp . s3://wbg-geography01/FATHOM/ --exclude "*" --include "*.vrt"  --recursive

# Inspect copied VRTs

In [53]:
s3_bucket = 'wbg-geography01'
s3_prefix = 'FATHOM/'

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(s3_bucket)

for o in my_bucket.objects.filter(Prefix=s3_prefix):
    if o.key.endswith(".vrt"):
        print(o.key)


FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP1_2.6-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP2_4.5-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP5_8.5-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2050-SSP1_2.6-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2050-SSP2_4.5-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2050-SSP3_7.0-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2050-SSP5_8.5-PERCENTILE50-v3.0.vrt
FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10

KeyboardInterrupt: 

In [56]:
dir(dMisc)

['Config',
 'UNSIGNED',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'aws_search_ntl',
 'boto3',
 'get_geoboundaries',
 'gpd',
 'json',
 'urllib']

# DEBURGGING

In [3]:
# Generate a list of files for gdalbuildvrt
folder = "/home/wb411133/temp/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0"
all_files = [f'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/{x}' for x in os.listdir(folder)]
all_files


['v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00w177.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00e007.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00w089.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00e008.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00e128.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00w158.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00e006.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00e100.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00e113.tif',
 'v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n00e172.tif',
